In [17]:
import pandas as pd

import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(context = 'notebook', #mostly controls relative sizes of things on plot 
#                         #The base context is “notebook”, and the other contexts are “paper”, “talk”, and “poster”
#         style = 'darkgrid', #dict, None, or one of {darkgrid, whitegrid, dark, white, ticks}
#         palette = 'deep', # Should be something that color_palette() can process.
#         font_scale = 1, 
#         color_codes = False, 
#         rc = None)

# from IPython.core.interactiveshell import InteractiveShell 
# InteractiveShell.ast_node_interactivity = 'last_expr' 
# setting = "all" allows multiple outputs to be displayed for a given input cell. don't use w plotting!

from IPython.display import display

# %matplotlib notebook
#%matplotlib inline

pd.__version__ , np.__version__ #,  matplotlib.__version__, sns.__version__

('0.23.0', '1.14.3')

In [18]:
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, KFold, StratifiedKFold, \
                                    ShuffleSplit, GridSearchCV, RandomizedSearchCV

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

In [19]:
pwd

'/home/thedohn/Notebooks'

In [20]:
#cd '/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/'

In [21]:
from Home_Credit_package.master_pipeline import master_pipeline

# load this

## load df.

In [22]:
path = ''
original_cleaned_df = pd.read_csv(path + 'complete_initial_wrangled_data.csv', index_col = 'SK_ID_CURR')
level_1_metafeatures_df = pd.read_csv(path + 'level_2_input_FINAL_level_1_meta_features_df.csv', index_col = 'SK_ID_CURR')
total_df = pd.merge(original_cleaned_df, level_1_metafeatures_df, left_index=True, right_index=True, how = 'outer' )

In [23]:
total_df.info(verbose = True, null_counts = True);

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 91 columns):
TARGET                               307511 non-null int64
NAME_CONTRACT_TYPE                   307511 non-null object
CODE_GENDER                          307511 non-null object
FLAG_OWN_CAR                         307511 non-null object
FLAG_OWN_REALTY                      307511 non-null object
CNT_CHILDREN                         307511 non-null int64
NAME_EDUCATION_TYPE                  307511 non-null object
NAME_FAMILY_STATUS                   307511 non-null object
DAYS_BIRTH                           307511 non-null int64
FLAG_EMP_PHONE                       307511 non-null int64
FLAG_WORK_PHONE                      307511 non-null int64
FLAG_PHONE                           307511 non-null int64
FLAG_EMAIL                           307511 non-null int64
CNT_FAM_MEMBERS                      307511 non-null int64
REGION_RATING_CLIENT                 307511 non-null

In [24]:
# CHECK: this should be empty if everything is non null
total_df.isnull().any()[total_df.isnull().any()==True]

Series([], dtype: bool)

# models with only EXT SOURCES + level 1 final scores.                                     

In [25]:
minimal_feats = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
                 'pwr_rescale_RanFor_EXTpoly', 'pwr_rescale_RanFor_AllFeats', 
                 'pwr_rescale_LogReg_EXTpoly','pwr_rescale_LogReg_AllFeats', 
                 'pwr_rescale_MLP_AllFeats']

In [26]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df[minimal_feats],
                                                                                 int_cutoff=20, 
                                                                                 poly_deg=4, 
                                                                                 feats_with_interaction=[]
                                                                                 )

Total original feat len is 8. Sum of feats is 8.
Final array is length 40. final feature list is length 40.


### DONE. random forest grid search.

In [27]:
param_dist_dict =  { 'max_depth' :         [4,5,6,7],
                     'min_samples_leaf' :  [9,10,11,12,13],
                     'min_samples_split' : [2], 
                     'n_estimators':       [40],
                    }

forest_reg = RandomForestClassifier(random_state=0,
                                    class_weight = None)

my_grid_search = GridSearchCV(forest_reg, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_depth=4, min_samples_leaf=9, min_samples_split=2, n_estimators=40 
Memmaping (shape=(307511, 40), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080666248/3195-140473954781728-f9e19e0ef8d47f852076b18d96de02fe.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080666248/3195-140473954781728-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080666248/3195-140473954781728-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080666248/3195-140473954781728-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] max_depth=4, min_samples_leaf=9, min_samples_split=2, n_estimators=40 
[CV] max_depth=4, min_samples_leaf=9, min_samples_split=2, n_estimators=40 
Memmaping (shape=(205007,), dtype=int64) to new file /

[CV] max_depth=6, min_samples_leaf=10, min_samples_split=2, n_estimators=40 
[CV]  max_depth=5, min_samples_leaf=13, min_samples_split=2, n_estimators=40, score=0.7687559397224715, total=  33.5s
[CV] max_depth=6, min_samples_leaf=10, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:  2.2min
[CV]  max_depth=5, min_samples_leaf=12, min_samples_split=2, n_estimators=40, score=0.7707508207578276, total=  34.1s
[CV] max_depth=6, min_samples_leaf=10, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.2min
[CV]  max_depth=5, min_samples_leaf=13, min_samples_split=2, n_estimators=40, score=0.7653187530961645, total=  34.6s
[CV] max_depth=6, min_samples_leaf=11, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  2.2min
[CV]  max_depth=5, min_samples_leaf=13, min_samples_split=2, n_estimators=40, score=0.7707565226825928, total=  34.1s
[CV] max_depth=6, min_samples_leaf

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [4, 5, 6, 7], 'min_samples_leaf': [9, 10, 11, 12, 13], 'min_samples_split': [2], 'n_estimators': [40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [28]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'max_depth': 4, 'min_samples_leaf': 9, 'min_samples_split': 2, 'n_estimators': 40}",0.767948
1,"{'max_depth': 4, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 40}",0.767949
2,"{'max_depth': 4, 'min_samples_leaf': 11, 'min_samples_split': 2, 'n_estimators': 40}",0.767952
3,"{'max_depth': 4, 'min_samples_leaf': 12, 'min_samples_split': 2, 'n_estimators': 40}",0.767953
4,"{'max_depth': 4, 'min_samples_leaf': 13, 'min_samples_split': 2, 'n_estimators': 40}",0.767952
5,"{'max_depth': 5, 'min_samples_leaf': 9, 'min_samples_split': 2, 'n_estimators': 40}",0.768262
6,"{'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 40}",0.768257
7,"{'max_depth': 5, 'min_samples_leaf': 11, 'min_samples_split': 2, 'n_estimators': 40}",0.768257
8,"{'max_depth': 5, 'min_samples_leaf': 12, 'min_samples_split': 2, 'n_estimators': 40}",0.768274
9,"{'max_depth': 5, 'min_samples_leaf': 13, 'min_samples_split': 2, 'n_estimators': 40}",0.768277


In [29]:
my_grid_search.best_score_

0.7683421720049418

In [30]:
RF_optimized_parms = my_grid_search.best_params_
RF_optimized_parms

{'max_depth': 6,
 'min_samples_leaf': 11,
 'min_samples_split': 2,
 'n_estimators': 40}

In [31]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7723052463371459

In [32]:
del my_grid_search

### DONE. log reg grid search.

In [33]:
param_grid = [{ 'C' : [72,74,76,78,80,82,84,86,88] }]

my_LgRg = LogisticRegression(penalty= 'l2', 
                            random_state = 0, 
                            class_weight = None) 

my_LgRg_grid_search = GridSearchCV(my_LgRg, 
                           param_grid, 
                           cv = 3, # for integer inputs, Stratified folding is default. 
                           return_train_score = True , 
                           scoring = 'roc_auc',
                           n_jobs = -1, 
                           refit = True,
                           verbose = 51,
                            )

my_LgRg_grid_search.fit(total_df_piped, 
                        total_df['TARGET'])

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=72 ............................................................
Memmaping (shape=(307511, 40), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140474891485256/3195-140473954796152-f9e19e0ef8d47f852076b18d96de02fe.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474891485256/3195-140473954796152-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474891485256/3195-140473954796152-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474891485256/3195-140473954796152-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] C=72 ............................................................
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474891485256/3195-140473954796152-52b45483e9e8ae5

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [72, 74, 76, 78, 80, 82, 84, 86, 88]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [34]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_LgRg_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'C': 72},0.768990
1,{'C': 74},0.768990
2,{'C': 76},0.768990
3,{'C': 78},0.768991
4,{'C': 80},0.768991
5,{'C': 82},0.768990
6,{'C': 84},0.768987
7,{'C': 86},0.768985
8,{'C': 88},0.768991


In [35]:
my_LgRg_grid_search.best_score_

0.768991307672436

In [36]:
LgRg_optimized_parms = my_LgRg_grid_search.best_params_
LgRg_optimized_parms

{'C': 88}

In [37]:
train_scores = [x[1] for x in my_LgRg_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.769200101381084

In [38]:
del my_LgRg_grid_search

### DONE. MLP Classifier.

In [39]:
param_dist_dict =  { 'alpha' :         [.11,.13,.15],
                    'hidden_layer_sizes' : [(67, ),(70, ),(73, )],
                    }

In [40]:
my_MLP = MLPClassifier(random_state=0, 
                       tol=0.0001, 
                     #  verbose=51, 
                       warm_start=False, 
                       momentum=0.9)


my_grid_search = GridSearchCV(my_MLP, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] alpha=0.11, hidden_layer_sizes=(67,) ............................
Memmaping (shape=(307511, 40), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954782960/3195-140473954908144-f9e19e0ef8d47f852076b18d96de02fe.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954782960/3195-140473954908144-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954782960/3195-140473954908144-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954782960/3195-140473954908144-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] alpha=0.11, hidden_layer_sizes=(67,) ............................
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954782960/3195-140473954908144-52b45483e9e8ae5

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.11, 0.13, 0.15], 'hidden_layer_sizes': [(67,), (70,), (73,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [41]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'alpha': 0.11, 'hidden_layer_sizes': (67,)}",0.768867
1,"{'alpha': 0.11, 'hidden_layer_sizes': (70,)}",0.768771
2,"{'alpha': 0.11, 'hidden_layer_sizes': (73,)}",0.767385
3,"{'alpha': 0.13, 'hidden_layer_sizes': (67,)}",0.768863
4,"{'alpha': 0.13, 'hidden_layer_sizes': (70,)}",0.768772
5,"{'alpha': 0.13, 'hidden_layer_sizes': (73,)}",0.767374
6,"{'alpha': 0.15, 'hidden_layer_sizes': (67,)}",0.768933
7,"{'alpha': 0.15, 'hidden_layer_sizes': (70,)}",0.768624
8,"{'alpha': 0.15, 'hidden_layer_sizes': (73,)}",0.767456


In [42]:
my_grid_search.best_score_

0.7689326925827549

In [43]:
my_grid_search_optimized_parms = my_grid_search.best_params_
my_grid_search_optimized_parms

{'alpha': 0.15, 'hidden_layer_sizes': (67,)}

In [44]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7689057213274595

In [45]:
del my_grid_search

# models with all features

In [46]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df,
                                                                                 int_cutoff=20, 
                                                                                 poly_deg=4, 
                                                                                 feats_with_interaction=[]
                                                                                 )

Total original feat len is 90. Sum of feats is 90.
Final array is length 442. final feature list is length 442.


### DONE random forest grid search

In [66]:
param_dist_dict =  { 'max_depth' :         [14,16,18],
                     'min_samples_leaf' :  [546,548,550,552,554],
                     'min_samples_split' : [2], 
                     'n_estimators':       [40],
                    }

forest_reg = RandomForestClassifier(random_state=0,
                                    class_weight = None)

my_grid_search = GridSearchCV(forest_reg, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Memmaping (shape=(79337838,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140473954739424/3195-140473954730168-7425de812d76baf4d8183fd11603bedf.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140473954739424/3195-140473954730168-511e4e07e852e42372cd1047cfce8c24.pkl
Memmaping (shape=(79337838,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954739424/3195-140473954730168-36f0cabefc9e949ddca5595172ca52f7.pkl
[CV] max_depth=14, min_samples_leaf=546, min_samples_split=2, n_estimators=40 
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954739424/3195-140473954730168-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140473954739424/3195-140473954730168-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 10.7min
[CV]  max_depth=16, min_samples_leaf=552, min_samples_split=2, n_estimators=40, score=0.7650235232359144, total= 3.3min
[CV]  max_depth=16, min_samples_leaf=552, min_samples_split=2, n_estimators=40, score=0.7687193957229415, total= 3.4min
[CV] max_depth=18, min_samples_leaf=546, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 13.8min
[CV] max_depth=18, min_samples_leaf=548, min_samples_split=2, n_estimators=40 
[CV]  max_depth=16, min_samples_leaf=552, min_samples_split=2, n_estimators=40, score=0.770734763414368, total= 3.3min
[CV] max_depth=18, min_samples_leaf=548, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed: 13.9min
[CV]  max_depth=16, min_samples_leaf=554, min_samples_split=2, n_estimators=40, score=0.7684036129889776, total= 3.3min
[Parallel(n_jobs=-1)]: Done  28 task

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [14, 16, 18], 'min_samples_leaf': [546, 548, 550, 552, 554], 'min_samples_split': [2], 'n_estimators': [40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [67]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'max_depth': 14, 'min_samples_leaf': 546, 'min_samples_split': 2, 'n_estimators': 40}",0.767852
1,"{'max_depth': 14, 'min_samples_leaf': 548, 'min_samples_split': 2, 'n_estimators': 40}",0.767898
2,"{'max_depth': 14, 'min_samples_leaf': 550, 'min_samples_split': 2, 'n_estimators': 40}",0.768070
3,"{'max_depth': 14, 'min_samples_leaf': 552, 'min_samples_split': 2, 'n_estimators': 40}",0.768069
4,"{'max_depth': 14, 'min_samples_leaf': 554, 'min_samples_split': 2, 'n_estimators': 40}",0.767896
5,"{'max_depth': 16, 'min_samples_leaf': 546, 'min_samples_split': 2, 'n_estimators': 40}",0.767988
6,"{'max_depth': 16, 'min_samples_leaf': 548, 'min_samples_split': 2, 'n_estimators': 40}",0.768081
7,"{'max_depth': 16, 'min_samples_leaf': 550, 'min_samples_split': 2, 'n_estimators': 40}",0.768184
8,"{'max_depth': 16, 'min_samples_leaf': 552, 'min_samples_split': 2, 'n_estimators': 40}",0.768159
9,"{'max_depth': 16, 'min_samples_leaf': 554, 'min_samples_split': 2, 'n_estimators': 40}",0.767934


In [68]:
RF_optimized_parms = my_grid_search.best_params_
RF_optimized_parms

{'max_depth': 16,
 'min_samples_leaf': 550,
 'min_samples_split': 2,
 'n_estimators': 40}

In [69]:
my_grid_search.best_score_

0.7681837088411191

In [70]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7865378262227534

In [71]:
del my_grid_search

### DONE. log reg grid search

In [53]:
param_grid = [{ 'C' : [177,178,179,180,181] }]

my_LgRg = LogisticRegression(penalty= 'l2', 
                            random_state = 0, 
                            class_weight = None) 

my_LgRg_grid_search = GridSearchCV(my_LgRg, 
                           param_grid, 
                           cv = 3, # for integer inputs, Stratified folding is default. 
                           return_train_score = True , 
                           scoring = 'roc_auc',
                           n_jobs = -1, 
                           refit = True,
                           verbose = 51,
                            )

my_LgRg_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Memmaping (shape=(79337838,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140474938518104/3195-140473957821688-7425de812d76baf4d8183fd11603bedf.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140474938518104/3195-140473957821688-511e4e07e852e42372cd1047cfce8c24.pkl
Memmaping (shape=(79337838,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140474938518104/3195-140473957821688-36f0cabefc9e949ddca5595172ca52f7.pkl
[CV] C=177 ...........................................................
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474938518104/3195-140473957821688-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140474938518104/3195-140473957821688-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new 

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [177, 178, 179, 180, 181]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [54]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_LgRg_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'C': 177},0.771302
1,{'C': 178},0.771511
2,{'C': 179},0.771586
3,{'C': 180},0.771202
4,{'C': 181},0.771135


In [55]:
my_LgRg_grid_search.best_score_

0.7715860050402779

In [56]:
LgRg_optimized_parms = my_LgRg_grid_search.best_params_
LgRg_optimized_parms

{'C': 179}

In [57]:
train_scores = [x[1] for x in my_LgRg_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7738151103627916

In [58]:
del my_LgRg_grid_search

### DONE. MLP Classifier

In [72]:
param_dist_dict =  { 'alpha' :         [.04,.05,.06],
                    'hidden_layer_sizes' : [(52,),(55, ),(60, ),(65, )],
                    }

In [73]:
my_MLP = MLPClassifier(random_state=0, 
                       tol=0.0001, 
                     #  verbose=51, 
                       warm_start=False, 
                       momentum=0.9)


my_grid_search = GridSearchCV(my_MLP, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Memmaping (shape=(79337838,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140475080234376/3195-140473957820456-7425de812d76baf4d8183fd11603bedf.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3195_140475080234376/3195-140473957820456-511e4e07e852e42372cd1047cfce8c24.pkl
Memmaping (shape=(79337838,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080234376/3195-140473957820456-36f0cabefc9e949ddca5595172ca52f7.pkl
[CV] alpha=0.04, hidden_layer_sizes=(52,) ............................
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080234376/3195-140473957820456-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3195_140475080234376/3195-140473957820456-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new

[Parallel(n_jobs=-1)]: Done  29 out of  36 | elapsed: 52.2min remaining: 12.6min
[CV]  alpha=0.06, hidden_layer_sizes=(55,), score=0.7652482754755549, total=11.4min
[Parallel(n_jobs=-1)]: Done  30 out of  36 | elapsed: 53.8min remaining: 10.8min
[CV]  alpha=0.06, hidden_layer_sizes=(65,), score=0.7669415990785962, total= 6.6min
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed: 55.5min remaining:  8.9min
[CV]  alpha=0.06, hidden_layer_sizes=(65,), score=0.766134188937864, total= 6.5min
[Parallel(n_jobs=-1)]: Done  32 out of  36 | elapsed: 55.6min remaining:  6.9min
[CV]  alpha=0.06, hidden_layer_sizes=(60,), score=0.7650163862870678, total= 7.8min
[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 55.7min remaining:  5.1min
[CV]  alpha=0.06, hidden_layer_sizes=(60,), score=0.7686361896265752, total= 7.5min
[Parallel(n_jobs=-1)]: Done  34 out of  36 | elapsed: 56.3min remaining:  3.3min
[CV]  alpha=0.06, hidden_layer_sizes=(60,), score=0.763941586157705, total= 8.3min
[CV]  alpha=

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.04, 0.05, 0.06], 'hidden_layer_sizes': [(52,), (55,), (60,), (65,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [74]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'alpha': 0.04, 'hidden_layer_sizes': (52,)}",0.766230
1,"{'alpha': 0.04, 'hidden_layer_sizes': (55,)}",0.766293
2,"{'alpha': 0.04, 'hidden_layer_sizes': (60,)}",0.766001
3,"{'alpha': 0.04, 'hidden_layer_sizes': (65,)}",0.766186
4,"{'alpha': 0.05, 'hidden_layer_sizes': (52,)}",0.767126
5,"{'alpha': 0.05, 'hidden_layer_sizes': (55,)}",0.767896
6,"{'alpha': 0.05, 'hidden_layer_sizes': (60,)}",0.765753
7,"{'alpha': 0.05, 'hidden_layer_sizes': (65,)}",0.767066
8,"{'alpha': 0.06, 'hidden_layer_sizes': (52,)}",0.767219
9,"{'alpha': 0.06, 'hidden_layer_sizes': (55,)}",0.765901


In [75]:
my_grid_search.best_score_

0.7678962576420044

In [76]:
my_grid_search_optimized_parms = my_grid_search.best_params_
my_grid_search_optimized_parms

{'alpha': 0.05, 'hidden_layer_sizes': (55,)}

In [77]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7760116364383361

In [78]:
del my_grid_search